The code is designed to detect swimming pools in images using a VGG16-based model. The model predicts bounding boxes around the swimming pools, and its performance is evaluated on a test dataset.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import json
from matplotlib.patches import Rectangle
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

# Data
PATH = ''
NUMBER_TO_LOAD = 500  # for whole dataset, use 14964
MUTE = True
IMG_SIZE = (128, 128)
INPUT_SHAPE = (128, 128, 3)

with open(PATH + 'metadata.json') as f:
    y_dict = json.load(f)

count = 0
for i in y_dict.keys():
    if y_dict[i]['has_pool'] == False:
        count += 1
if count == 0:
    print("All images have swimming pools!")

png_name = []
for i in list(y_dict.keys())[:NUMBER_TO_LOAD]:
    try:
        img = plt.imread(PATH + 'images/' + i)
    except:
        if not MUTE:
            print(f"cannot find the image {i}")
    else:
        png_name.append(i)
print(f"{len(png_name)} images are valid out of {NUMBER_TO_LOAD}!")

X_train, X_test = train_test_split(png_name, test_size=0.2)
X_train, X_val = train_test_split(X_train, test_size=0.25)

# Image loading and preprocessing
def load_resize_img(y_dict, png_list, path=PATH, image_size=IMG_SIZE, plot=False):
    image = []
    label = []
    for i in png_list:
        img = plt.imread(PATH + 'images/' + i)
        image.append(cv2.resize(img, image_size))
        coord = np.array([[j['x'], j['y']] for j in y_dict[i]['bounds_x_y']], np.float32)
        x, y, w, h = cv2.boundingRect(coord)
        label.append([x, y, x + w, y + h])
    if plot:
        return pd.DataFrame({'png_name': png_list, 'image': image, 'label': label})
    else:
        df = pd.DataFrame({'image': image, 'label': label})
        X = np.stack([row['image'] for index, row in df.iterrows()])
        y = np.stack([list(map(lambda x: x / 512, row['label'])) for index, row in df.iterrows()])
        del df
        return X, y

def plot_img(df, n_to_plot=10):
    df_plot = df.loc[:n_to_plot, :]
    fig, axs = plt.subplots(1, n_to_plot, figsize=(80, 80 * n_to_plot))
    for i in range(n_to_plot):
        axs[i].imshow(df_plot.iloc[i, 1])
        xs, ys = zip(*[(i['x'], i['y']) for i in y_dict[df_plot.iloc[i, 0]]['bounds_x_y']])
        axs[i].plot(xs, ys, linewidth=5)
        x1, y1, x2, y2 = df_plot.iloc[i, 2]
        axs[i].add_patch(Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=5, edgecolor='r', facecolor='none'))
        axs[i].axis("off")
    plt.show()

df = load_resize_img(y_dict, X_train, path=PATH, image_size=(512, 512), plot=True)
plot_img(df, n_to_plot=10)

# Model Building
class VGG_detection():
    def __init__(self, params, verbose=False):
        self.params = params
        self.verbose = verbose

    def get_vgg(self):
        input_shape = self.params['input_shape']
        weights = self.params['weights']
        include_top = self.params['include_top']
        vgg = VGG16(weights=weights, include_top=include_top, input_tensor=Input(shape=input_shape))
        vgg.trainable = False
        flatten = vgg.output
        flatten = Flatten()(flatten)
        x = Dense(128, activation="relu")(flatten)
        x = Dense(64, activation="relu")(x)
        x = Dense(32, activation="relu")(x)
        x = Dense(4, activation="sigmoid")(x)
        self.model = Model(inputs=vgg.input, outputs=x)
        return self

    # ... (rest of the class methods)

params = {
    'input_shape': INPUT_SHAPE,
    'weights': "imagenet",
    'include_top': False,
    'optimizer': "adam",
    'loss': "mse",
    'monitor': "mean_squared_error",
    'save_name': False,
    'batch_size': 4,
    'epochs': 20,
    'load_name': False
}

X_train, y_train = load_resize_img(y_dict, X_train, path=PATH, image_size=IMG_SIZE, plot=False)
X_val, y_val = load_resize_img(y_dict, X_val, path=PATH, image_size=IMG_SIZE, plot=False)

vgg = VGG_detection(params, MUTE).get_vgg()
vgg.fit_vgg(X_train, X_val, y_train, y_val)
vgg.learning_curves()

# Prediction
X_test, y_test = load_resize_img(y_dict, X_test, path=PATH, image_size=IMG_SIZE, plot=False)
y_pred = vgg.predict(X_test)
vgg.show_result(5, X_test, y_test, y_pred, best=True)
vgg.show_result(5, X_test, y_test, y_pred, best=False)

vgg.model.summary()
